# An introduction to LoRA and QLoRA



In [1]:
# pip installs
# peft stend for paremter efficent fine tunning

!pip install -q datasets==2.21.0 requests torch peft bitsandbytes transformers==4.43.1 trl accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [2]:
# imports
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [3]:
# Constants
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"Meziane/pricer-one-29-11-2024"


# Hyperparameters for QLoRA Fine-Tuning
LORA_R = 32
LORA_ALPHA = 64  # alpha double R

# these are the moudel I am targetting and these layer, thera are the most common layer to taget
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

In [4]:
# Log in to HuggingFace
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Different Quantization



In [4]:
# Load the Base Model without quantization
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")  # device map means usign the gpu

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

Memory footprint: 32.1 GB


In [6]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
    (rotary_emb

In [6]:
# Load the Base Model using 8 bit

# firts ## Restart your session

# The package "Bits and Bytes" is used for quantization in models, and it provides a class called BitsAndBytesConfig.
# In this class, you specify the type of quantization configuration and the level of precision reduction for your model.
# Here, we set the 'load_in_8bit' parameter to True, which means we are reducing the model's precision from 32 bits to 8 bits.
# This is done to load the base model with reduced precision, which helps in saving memory and improving performance.
# We pass this configuration as a parameter when loading the base Llama 3.1 model.
# This technique of precision reduction (from 32 bits to 8 bits) was done in previous work, but we didn't discuss it much before.


quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

Memory footprint: 9.1 GB


Quantization reduces weight precision without changing the architecture,
keeping performance largely unaffected.
The model size drops from 32GB to 9GB with minimal impact.


In [8]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


-Architecture Overview

Embedding Layer:
Tokens are embedded into vectors based on the number of tokens and the size of the embedding vectors.

32 Llama Decoder Layers:
Each consists of attention layers (Q, K, V, O projections) that process the input. These layers have varying dimensions, typically with 4000 dimensions in and out, and 1000 out in some cases.

Multi-Layer Perceptron (MLP):
The MLP layer upsamples and downscales dimensions, followed by an activation function (SELU) for non-linearity.

Layer Normalization:
Normalizes inputs across layers for stability.

LM Head:
The final linear layer used for generating outputs. It’s often targeted for tasks like format conversion or language generation.

Target Modules:
Attention layers (Q, K, V, O projections) are commonly targeted, but the LM head can also be if generating specific outputs (e.g., JSON or different languages).

Key Takeaway:
The architecture, with its attention layers and LM head, allows flexibility for tasks like fine-tuning and output generation, which can be customized using Lora adapters

## Notes for the below code

In this case, we're using four-bit quantization. There are a few other important settings, such as use double quant, which performs two passes of quantization. This reduces memory usage by 10-20%, with minimal impact on the neural network's power, making it almost a free optimization. It's recommended to set use double quant to true.

The compute dtype setting defines the data type for computations. Using bfloat16 (binary float 16) is commonly recommended because it speeds up training with only a slight trade-off in training quality. In my experience, this leads to faster training without noticeable changes in optimization performance.

Next, the four-bit quant type specifies how the reduced precision (4-bit) numbers should be interpreted. A common approach is to map these values to a floating point number using an nf4 distribution, which I’ve found to be the most effective. While this is the standard setting, it's still a hyperparameter you can experiment with.

Using these configurations, the model's memory footprint is reduced to an impressive 5.6GB from

In [7]:
# Load the Tokenizer and the Base Model using 4 bit

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

Memory footprint: 5.59 GB


In [9]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

# Note for the code below

This example demonstrates a model with a reduced memory footprint. The model size is only slightly different from the previous one, with a 100MB difference, bringing the memory usage closer to a more manageable size. The architecture of this model includes 32 layers of the Lama decoder, followed by attention layers with projections (Q, K, V, O). Each of these projections has base layers along with Lora A and Lora B matrices, with 32 dimensions for the rank (R).

These Lora A and Lora B matrices help adjust the base layers with minimal changes. This setup reduces the model's size by using lower-dimensional matrices, making it more efficient while retaining the model's performance. The parameters from these adapters total around 27 million, with a size of approximately 109MB. This is much smaller compared to the full Lama model, which has 8 billion parameters and requires 32GB of memory.

The model architecture remains mostly unchanged except for these adjustments. The final size of the Lora adapter is 109MB, which is confirmed when looking at the saved parameters in Hugging Face. This shows how Q Laura enables fine-tuning with fewer parameters, allowing for efficient memory usage while still achieving the desired model performance.

In [ ]:
# load fine tunnign model and as you can see we use a peft model nto autocasullm

fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

In [ ]:
# Each of the Target Modules has 2 LoRA Adaptor matrices, called lora_A and lora_B
# These are designed so that weights can be adapted by adding alpha * lora_A * lora_B
# Let's count the number of weights using their dimensions:

# See the matrix dimensions above
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# Each layer comes to
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# There are 32 layers
params = lora_layer * 32

# So the total size in MB is
size = (params * 4) / 1_000_000

print(f"Total number of params: {params:,} and size {size:,.1f}MB")